# PIC-SURE API tutorial using Cure SC database

This is a tutorial notebook, aimed for the user to be quickly up and running with the python PIC-SURE API. It covers the main functionalities of the API.

## PIC-SURE python API 
### What is PIC-SURE? 

PIC-SURE stands for Patient-centered Information Commons: Standardized Unification of Research Elements. Original data exposed through PIC-SURE API encompasses a large heterogeneity of data organization underneath. PIC-SURE hides this complexity and exposes the different study datasets in a single tabular format. By simplifying the process of data extraction, it allows investigators to focus on the downstream analyses and to facilitate reproducible sciences.

### More about PIC-SURE
The API is available in two different programming languages, python and R, allowing investigators to query datasets in the same way using either of language. The R/python PIC-SURE API is a small part of the entire PIC-SURE platform.

The API is actively developed by the Avillach Lab at Harvard Medical School.

GitHub repo:
* https://github.com/hms-dbmi/pic-sure-python-adapter-hpds
* https://github.com/hms-dbmi/pic-sure-python-client


 -------   

# Getting your own user-specific security token

**Before running this notebook, please be sure you have [added your security token](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/Cure_Sickle_Cell#get-your-security-token). This documentation contains an explanation about how to get a security token, which is required to access the databases.**

# Environment set-up

### Pre-requisite
- python 3.6 or later
- pip: python package manager, already available in most system with a python interpreter installed ([pip installation instructions](https://pip.pypa.io/en/stable/installing/))

### IPython Magic command

The following code loads the `autoreload` IPython extension. Although `autoreload` is not necessary to execute the rest of the notebook, it enables the notebook to reload every dependency each time python code is executed.  This will allow the notebook to take into account changes in imported external files, such as the user defined functions stored in separate file, without needing to manually reload libraries. This is helpful when developing interactively. Learn more about [IPython Magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html).

In [1]:
%load_ext autoreload
%autoreload 2

### Install python packages

Using the pip package manager, we install the packages listed in the `requirements.txt` file.

In [2]:
!cat requirements.txt # List contents of the requirements.txt file

numpy>=1.17.3
matplotlib>=3.1.1
pandas>=0.25.3
scipy>=1.3.1
tqdm>=4.38.0
statsmodels>=0.10.2
git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
git+https://github.com/hms-dbmi/pic-sure-python-client.git 


In [3]:
import sys
!{sys.executable} -m pip install -r requirements.txt

  Cloning https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git to /tmp/pip-req-build-jmdgx5aa
  Running command git clone -q https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git /tmp/pip-req-build-jmdgx5aa
  Cloning https://github.com/hms-dbmi/pic-sure-python-client.git to /tmp/pip-req-build-u6k6jo22
  Running command git clone -q https://github.com/hms-dbmi/pic-sure-python-client.git /tmp/pip-req-build-u6k6jo22
     |████████████████████████████████| 75 kB 4.9 MB/s  eta 0:00:01
     |████████████████████████████████| 95 kB 5.0 MB/s  eta 0:00:01
  Created wheel for PicSureClient: filename=PicSureClient-0.1.0-py2.py3-none-any.whl size=10225 sha256=476d0611c62780c1fa479962376f601055a94937f0ca2ce374497e4f40aea0ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-5xamjbee/wheels/31/ef/21/e362bba8de04e0072fafec9f77bd1abdf7e166213d27e98729
  Created wheel for PicSureHpdsLib: filename=PicSureHpdsLib-0.9.0-py2.py3-none-any.whl size=21999 sha256=50dd3eb6e9d0994e09465646a9f999f19

Import external dependencies and user-defined functions stored in the `python_lib` folder.

In [4]:
import json
from pprint import pprint

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from scipy import stats

import PicSureHpdsLib
import PicSureClient

from python_lib.utils import get_multiIndex_variablesDict, get_dic_renaming_vars, joining_variablesDict_onCol

In [5]:
print("NB: This Jupyter Notebook has been written using PIC-SURE API following versions:\n- PicSureClient: 0.1.0\n- PicSureHpdsLib: 1.1.0\n")
print("The PIC-SURE API libraries versions you've been downloading are: \n- PicSureClient: {0}\n- PicSureHpdsLib: {1}".format(PicSureClient.__version__, PicSureHpdsLib.__version__))

NB: This Jupyter Notebook has been written using PIC-SURE API following versions:
- PicSureClient: 0.1.0
- PicSureHpdsLib: 1.1.0

The PIC-SURE API libraries versions you've been downloading are: 
- PicSureClient: 1.1.0
- PicSureHpdsLib: 1.1.0


##### Set up the options for displaying tables and plots in this notebook

In [6]:
# Pandas DataFrame display options
pd.set_option("max.rows", 100)

# Matplotlib parameters options
fig_size = plt.rcParams["figure.figsize"]
 
# Prints: [8.0, 6.0]
fig_size[0] = 14
fig_size[1] = 8
plt.rcParams["figure.figsize"] = fig_size

font = {'weight' : 'bold',
        'size'   : 14}

plt.rc('font', **font)

### Connecting to a PIC-SURE network

You will need the following information before connecting to the PIC-SURE network:
* resource ID: ID of the resource that you are trying to access. You can leave the default value for this project.
* user-specific token text file: A text file called `token.txt` should contain the token retrieved from your user profile in PIC-SURE UI. This file needs to be located at the python root folder.

In [7]:
resource_id = "57e29a43-38c3-4c4b-84c9-dda8138badbe"
token_file = "token.txt"
PICSURE_network_URL = "https://curesc.hms.harvard.edu/picsure"

In [8]:
with open(token_file, "r") as f:
    my_token = f.read()

In [9]:
client = PicSureClient.Client()
connection = client.connect(PICSURE_network_URL, my_token, True)



+=========================================================================================+
|        [ WARNING ] you are specifying that you WANT to allow self-signed SSL            |
|        certificates to be acceptable for connections.  This may be useful for           |
|        working in a development environment or on systems that host public              |
|        data.  BEST SECURITY PRACTICES ARE THAT IF YOU ARE WORKING WITH SENSITIVE        |
|        DATA THEN ALL SSL CERTS BY THOSE EVIRONMENTS SHOULD NOT BE SELF-SIGNED.          |
+=========================================================================================+

+--------------------------------------+------------------------------------------------------
|  Resource UUID                       |  Resource Name                                  
+--------------------------------------+------------------------------------------------------
| 57e29a43-38c3-4c4b-84c9-dda8138badbe
+---------------------------------

In [10]:
adapter = PicSureHpdsLib.Adapter(connection)
resource = adapter.useResource(resource_id)

Two objects were created: a `connection` and a `resource` object, using the `picsure` and `hpds` libraries, respectively. 

Since will only be using a single resource, **the `resource` object is the only one we will need to proceed with this data analysis.** It should be noted that the `connection` object is useful to access different databases stored in different resources. 

The `resource` object is connected to the specific resource ID and enables us to query and retrieve data from this source.

### Getting help with the PIC-SURE python API

The `help()` method prints out the helper message for any PIC-SURE library function. For example, we can learn more about getting a resource using the following code:

In [11]:
resource.help()


        [HELP] PicSureHpdsLib.useResource(resource_uuid)
            .dictionary()       Used to access data dictionary of the resource
            .query()            Used to query against data in the resource
            .retrieveQueryResults(query_uuid) returns the results of an asynchronous query that has already been submitted to PICSURE

        [ENVIRONMENT]
              Endpoint URL: https://curesc.hms.harvard.edu/picsure/
             Resource UUID: 57e29a43-38c3-4c4b-84c9-dda8138badbe


This output tells us about the methods and functions of the `resource` object. 

## Using the *variables dictionary*

Once a connection to the desired resource has been established, we first need to get an understanding of which variables are available in the database. We will use the `dictionary` method of the `resource` object to do this.

A `dictionary` instance retrieves records that match a specific term. The `find()` method can be used to retrieve information about the available variables. For instance, looking for variables containing the term 'Sex' is done this way: 

In [12]:
dictionary = resource.dictionary()
dictionary_search = dictionary.find("Sex")
dictionary_search.DataFrame().head()

,categorical,categoryValues,observationCount,patientCount,HpdsDataType
KEY,,,,,
\CIBMTR - Cure Sickle Cell Disease\1 - Patient Related\Sex\,True,"[Female, Male]",1518,1518,phenotypes


Objects created by the `dictionary.find` method can expose the search results using 4 different methods: `.count()`, `.keys()`, `.entries()`, and `.DataFrame()`. 

In [13]:
pprint({"Count": dictionary_search.count(), 
        "Keys": dictionary_search.keys()[0:3],
        "Entries": dictionary_search.entries()[0:3]})

{'Count': 1,
 'Entries': [{'HpdsDataType': 'phenotypes',
              'categorical': True,
              'categoryValues': ['Female', 'Male'],
              'name': '\\CIBMTR - Cure Sickle Cell Disease\\1 - Patient '
                      'Related\\Sex\\',
              'observationCount': 1518,
              'patientCount': 1518}],
 'Keys': ['\\CIBMTR - Cure Sickle Cell Disease\\1 - Patient Related\\Sex\\']}


**The `.DataFrame()` method enables us to get the result of the dictionary search in a pandas DataFrame format. This way, it allows us to:** 


* Use the various information exposed in the dictionary (patient count, variable type ...) as criteria for variable selection.
* Use the row names of the DataFrame to get the actual variable names to be used in the query, as shown below.

Variable names aren't very pratical to use right away for two reasons:
1. Very long
2. Presence of backslashes that prevent copy-pasting. 

However, retrieving the dictionary search result in the form of a dataframe can help access the variable names.

Let's say we want to retrieve every variable for in the form of a DataFrame. We can do this using the code below:

In [15]:
plain_variablesDict = resource.dictionary().find().DataFrame()

Using the `dictionary.find()` function without arguments returns every entry, as shown in the help documentation.

In [16]:
resource.dictionary().help()


        [HELP] PicSureHpdsLib.Client(connection).useResource(uuid).dictionary()
            .find()                 Lists all data dictionary entries
            .find(search_string)    Lists matching data dictionary entries
        


In [17]:
plain_variablesDict.iloc[10:20,:]

,min,categorical,observationCount,patientCount,max,HpdsDataType,categoryValues
KEY,,,,,,,
\CIBMTR - Cure Sickle Cell Disease\5 - CRF data collection track only\Hemoglobin pre-conditioning unit\,NaN,True,732,732,NaN,phenotypes,"[Not reported, g/dL]"
\CIBMTR - Cure Sickle Cell Disease\5 - CRF data collection track only\Acute renal failure requiring dialysis\,NaN,True,732,732,NaN,phenotypes,"[No, Not reported, Yes]"
\CIBMTR - Cure Sickle Cell Disease\1 - Patient Related\HCT-comorbidity index\,NaN,True,1518,1518,NaN,phenotypes,"[0-2, 3+, Not collected before 2008]"
"\CIBMTR - Cure Sickle Cell Disease\4 - Outcomes\Death without acute graft versus host disease, grades II-IV\",NaN,True,1518,1518,NaN,phenotypes,"[No, Not Reported, Yes]"
\CIBMTR - Cure Sickle Cell Disease\5 - CRF data collection track only\Stroke pre-conditioning\,NaN,True,732,732,NaN,phenotypes,"[No, Not reported, Yes]"
"\CIBMTR - Cure Sickle Cell Disease\5 - CRF data collection track only\Time from HCT to intubation, months\",0.131579,False,34,34,200.032895,phenotypes,None
\CIBMTR - Cure Sickle Cell Disease\3 - Transplant Related\Year of transplant ( Grouped )\,NaN,True,1518,1518,NaN,phenotypes,"[2008-2012, 2013-2017, 2018-2019, < 2008]"
\CIBMTR - Cure Sickle Cell Disease\3 - Transplant Related\Donor-recipient HLA matching\,NaN,True,1518,1518,NaN,phenotypes,"[7/8, 8/8, <=6/8]"
\CIBMTR - Cure Sickle Cell Disease\5 - CRF data collection track only\Therapy given for growth hormone deficiency\,NaN,True,732,732,NaN,phenotypes,"[N/A, No, Not reported, Yes]"


### Export Full Data Dictionary to CSV

In order to export the data dictionary first we will create a Pandas dataframe called `fullVariableDict`.

In [18]:
fullVariableDict = resource.dictionary().find().DataFrame()

Let's make sure that `fullVariableDict` dataframe contains some values.

In [19]:
fullVariableDict.iloc[0:3,:]

,min,categorical,observationCount,patientCount,max,HpdsDataType,categoryValues
KEY,,,,,,,
"\CIBMTR - Cure Sickle Cell Disease\5 - CRF data collection track only\Time from HCT to diabetes, months\",0.131579,False,48,48,46.315789,phenotypes,None
\CIBMTR - Cure Sickle Cell Disease\5 - CRF data collection track only\Serum creatinine pre-conditioning unit\,NaN,True,732,732,NaN,phenotypes,"[Not reported, mg/dL]"
\CIBMTR - Cure Sickle Cell Disease\5 - CRF data collection track only\Hypertension (HTN) requiring therapy\,NaN,True,732,732,NaN,phenotypes,"[No, Not reported, Yes]"


In [20]:
fullVariableDict.to_csv('data_dictionary.csv')

You should now see a data_dictionary.csv in the Jupyter Hub file explorer.

#### Variable dictionary + pandas multiIndex

We can use a simple user-defined function (`get_multiIndex_variablesDict`) to add a little more information to the variable dictionary and to simplify working with variables names. It takes advantage of pandas MultiIndex functionality [see pandas official documentation on this topic](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html).

Although not an official feature of the API, such functionality illustrates how to quickly select groups of related variables.

Printing the multiIndexed variable Dictionary allows us to quickly see the tree-like organization of the variable names. Moreover, original and simplified variable names are now stored respectively in the `name` and `simplified_name` columns (simplified variable names is simply the last component of the variable name, which is usually the most informative to let us know what each variable is about).

In [21]:
variablesDict = get_multiIndex_variablesDict(plain_variablesDict)

In [22]:
variablesDict.loc[["CIBMTR - Cure Sickle Cell Disease"],:]

simplified_name  \
level_0                           level_1                            level_2                                                                                                 
CIBMTR - Cure Sickle Cell Disease 1 - Patient Related                Cases from 2016 Blood publication                                   Cases from 2016 Blood publication   
                                                                     Cases from 2019 Lancet Heamatology publication         Cases from 2019 Lancet Heamatology publication   
                                                                     Country of HCT institution                                                 Country of HCT institution   
                                                                     Ethnicity                                                                                   Ethnicity   
                                                                     HCT-comorbidity index                                                           HCT-comorbidity index   
...                                                                                                                                                                    ...   
                                  5 - CRF data collection track only Type of arrythmia                                                                   Type of arrythmia   
                                                                     Type of therapy for iron overload                                   Type of therapy for iron overload   
                                                                     VOD post-HCT                                                                             VOD post-HCT   
                                                                     Vaso-occlusive cirsis requiring hospitalization...  Vaso-occlusive cirsis requiring hospitalizatio...   
                                                                     Vaso-occlusive pain post HCT                                             Vaso-occlusive pain post HCT   

                                                                                                                                                                      name  \
level_0                           level_1                            level_2                                                                                                 
CIBMTR - Cure Sickle Cell Disease 1 - Patient Related                Cases from 2016 Blood publication                   \CIBMTR - Cure Sickle Cell Disease\1 - Patient...   
                                                                     Cases from 2019 Lancet Heamatology publication      \CIBMTR - Cure Sickle Cell Disease\1 - Patient...   
                                                                     Country of HCT institution                          \CIBMTR - Cure Sickle Cell Disease\1 - Patient...   
                                                                     Ethnicity                                           \CIBMTR - Cure Sickle Cell Disease\1 - Patient...   
                                                                     HCT-comorbidity index                               \CIBMTR - Cure Sickle Cell Disease\1 - Patient...   
...                                                                                                                                                                    ...   
                                  5 - CRF data collection track only Type of arrythmia                                   \CIBMTR - Cure Sickle Cell Disease\5 - CRF dat...   
                                                                     Type of therapy for iron overload                   \CIBMTR - Cure Sickle Cell Disease\5 - CRF dat...   
                                                                     VOD post-HCT                                        \CIBMTR - Cure Sickle Cell Dise

In [23]:
# Limit the number of lines to be displayed for the future outputs
pd.set_option("max.rows", 50)

Below is a simple example to illustrate the ease of use a multiIndex dictionary. Let's say we are interested in the variable called "5 - CRF data collection track only".

In [24]:
mask_study = variablesDict.index.get_level_values(0) == "CIBMTR - Cure Sickle Cell Disease"
mask_dctrack = variablesDict.index.get_level_values(1) == "5 - CRF data collection track only"
dctrack_variables = variablesDict.loc[mask_study & mask_dctrack,:]
dctrack_variables

simplified_name  \
level_0                           level_1                            level_2                                                                                                 
CIBMTR - Cure Sickle Cell Disease 5 - CRF data collection track only Acute chest syndrome (ACS) pre-conditioning               Acute chest syndrome (ACS) pre-conditioning   
                                                                     Acute chest syndrome post HCT                                           Acute chest syndrome post HCT   
                                                                     Acute renal failure requiring dialysis                         Acute renal failure requiring dialysis   
                                                                     Anxiety requiring therapy                                                   Anxiety requiring therapy   
                                                                     Arrhythmia                                                                                 Arrhythmia   
...                                                                                                                                                                    ...   
                                                                     Type of arrythmia                                                                   Type of arrythmia   
                                                                     Type of therapy for iron overload                                   Type of therapy for iron overload   
                                                                     VOD post-HCT                                                                             VOD post-HCT   
                                                                     Vaso-occlusive cirsis requiring hospitalization...  Vaso-occlusive cirsis requiring hospitalizatio...   
                                                                     Vaso-occlusive pain post HCT                                             Vaso-occlusive pain post HCT   

                                                                                                                                                                      name  \
level_0                           level_1                            level_2                                                                                                 
CIBMTR - Cure Sickle Cell Disease 5 - CRF data collection track only Acute chest syndrome (ACS) pre-conditioning         \CIBMTR - Cure Sickle Cell Disease\5 - CRF dat...   
                                                                     Acute chest syndrome post HCT                       \CIBMTR - Cure Sickle Cell Disease\5 - CRF dat...   
                                                                     Acute renal failure requiring dialysis              \CIBMTR - Cure Sickle Cell Disease\5 - CRF dat...   
                                                                     Anxiety requiring therapy                           \CIBMTR - Cure Sickle Cell Disease\5 - CRF dat...   
                                                                     Arrhythmia                                          \CIBMTR - Cure Sickle Cell Disease\5 - CRF dat...   
...                                                                                                                                                                    ...   
                                                                     Type of arrythmia                                   \CIBMTR - Cure Sickle Cell Disease\5 - CRF dat...   
                                                                     Type of therapy for iron overload                   \CIBMTR - Cure Sickle Cell Disease\5 - CRF dat...   
                                                                     VOD post-HCT                                        \CIBMTR - Cure Sickle Cell Dise

This simple filter can be easily combined with other filters to quickly select variables of interest.

In [36]:
variablesDict[variablesDict.categorical == False]

simplified_name  \
level_0                           level_1                            level_2                                                                                                 
CIBMTR - Cure Sickle Cell Disease 1 - Patient Related                Patient age at transplant, years                                     Patient age at transplant, years   
                                  3 - Transplant Related             Transplant number                                                                   Transplant number   
                                                                     Year of transplant                                                                 Year of transplant   
                                  4 - Outcomes                       Time from HCT to acute graft-vs-host disease, m...  Time from HCT to acute graft-vs-host disease, ...   
                                                                     Time from HCT to chronic graft-vs-host disease,...  Time from HCT to chronic graft-vs-host disease...   
                                                                     Time from HCT to date of last contact or death,...  Time from HCT to date of last contact or death...   
                                                                     Time from HCT to graft failure, months                         Time from HCT to graft failure, months   
                                                                     Time from HCT to neutrophil engraftment, months       Time from HCT to neutrophil engraftment, months   
                                                                     Time from HCT to platelet recovery, months                 Time from HCT to platelet recovery, months   
                                                                     Time from HCT to second malignancy, months                 Time from HCT to second malignancy, months   
                                  5 - CRF data collection track only Hemoglobin pre-conditioning                                               Hemoglobin pre-conditioning   
                                                                     Normal upper limit of serum albumin for center         Normal upper limit of serum albumin for center   
                                                                     Normal upper limit of serum creatinine for center   Normal upper limit of serum creatinine for center   
                                                                     Number of ACS syndromes within 2 years pre-HCT         Number of ACS syndromes within 2 years pre-HCT   
                                                                     Serum albumin pre-conditioning                                         Serum albumin pre-conditioning   
                                                                     Serum creatinine pre-conditioning                                   Serum creatinine pre-conditioning   
                                                                     Time from HCT to CNS hemorrhage, months                       Time from HCT to CNS hemorrhage, months   
                                                                     Time from HCT to Cataracts, months                                 Time from HCT to Cataracts, months   
                                                                     Time from HCT to DVT, months                                             Time from HCT to DVT, months   
                                                                     Time from HCT to HTN, months                                             Time from HCT to HTN, months   
                                                                     Time from HCT to PTSD, months                                           Time from HCT to PTSD, months   
                                                                     Time from HCT to TMA, months                                             Time from H

## Querying and retrieving data

The second cornerstone of the API is the `query` object, which is how we retrieve data from the resource.

The query object has several methods that enable us to build a query:

| Method | Arguments / Input | Output|
|--------|-------------------|-------|
| query.select.add() | variable names (string) or list of strings | all variables included in the list (no record subsetting)|
| query.require.add() | variable names (string) or list of strings | all variables; only records that do not contain null values for input variables |
| query.anyof.add() | variable names (string) or list of strings | all variables; only records that contain at least one non-null value for input variables |
| query.filter.add() | variable name and additional filtering values | input variable; only records that match filter criteria |

All 4 methods can be combined when building a query. The record eventually returned by the query has to meet all the different specified filters.

### Building the query

Let's say we are interested in the age at which patients from the following cohort received their transplant:
* males
* patients with avascular necrosis
* patients that received their transplant after the year 1999

First we will find variables pertaining to sex and avascular necrosis. We can do this by searching for "Sex" and "Avascular necrosis" in the `simplified_name` column of `variablesDict`.

In [48]:
sex_var = variablesDict.loc[variablesDict["simplified_name"] == "Sex", "name"].values[0]

avascular_necrosis_varname = variablesDict.loc[variablesDict["simplified_name"] == "Avascular necrosis", "name"].values[0]


In [50]:
# Peek at the result for avascular necrosis
variablesDict.loc[variablesDict["simplified_name"] == "Avascular necrosis", "name"] 

level_0                            level_1                             level_2           
CIBMTR - Cure Sickle Cell Disease  5 - CRF data collection track only  Avascular necrosis    \CIBMTR - Cure Sickle Cell Disease\5 - CRF dat...
Name: name, dtype: object

Next, we can find the variable pertaining to "Year of transplant".

In [51]:
yr_transplant_varname = variablesDict.loc[variablesDict["simplified_name"] == "Year of transplant", "name"].values[0]

Now we can create a new query and apply our filters to retrieve the cohort of interest.

In [58]:
my_query = resource.query()
# Patients with avascular necrosis
my_query.select().add(avascular_necrosis_varname)
my_query.filter().add(avascular_necrosis_varname, "Yes")

In [59]:
# Males
my_query.select().add(sex_var)
my_query.filter().add(sex_var, "Male")

In [60]:
# Patients receiving transplants after 1999
my_query.select().add(yr_transplant_varname)
my_query.filter().add(yr_transplant_varname, min=2000)

Now that we have narrowed down to males with avascular necrosis, we can add the variable of interest: "Patient age at transplant, years"

In [61]:
age_transplant_var = variablesDict.loc[variablesDict["simplified_name"] == "Patient age at transplant, years", "name"].values[0]
my_query.select().add(age_transplant_var)

## Retrieving the data

Once our query object is finally built, we use the `query.run()` function to retrieve the data corresponding to our query

In [62]:
query_df = my_query.getResultsDataFrame().set_index("Patient ID")

In [63]:
query_df

,"\CIBMTR - Cure Sickle Cell Disease\1 - Patient Related\Patient age at transplant, years\",\CIBMTR - Cure Sickle Cell Disease\1 - Patient Related\Sex\,\CIBMTR - Cure Sickle Cell Disease\3 - Transplant Related\Year of transplant\,\CIBMTR - Cure Sickle Cell Disease\5 - CRF data collection track only\Avascular necrosis\
Patient ID,,,,
42,15.0,Male,2007.0,Yes
295,18.0,Male,2008.0,Yes
336,13.0,Male,2009.0,Yes
612,16.0,Male,2012.0,Yes
725,20.0,Male,2014.0,Yes
752,15.0,Male,2014.0,Yes
766,13.0,Male,2014.0,Yes
772,5.0,Male,2014.0,Yes
895,11.0,Male,2015.0,Yes
